In [3]:
import os

import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "gestion_immo.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from agence.django import Utilisateur
from agence.models import Vendeur


vendeur = Utilisateur()
# vendeur

RuntimeError: Conflicting 'lieu' models in application 'agence': <class 'agence.models.Lieu'> and <class 'agence.django.Lieu'>.

In [4]:
from agence.models import Acheteur, Agent, Utilisateur, Vendeur

utilisateur = Utilisateur.objects.create(
    nom="Bambelle", prenom="Larry", telephone="+33158350027", email="bambelle.larry@youhou.fr"
)
# acheteur = Acheteur.objects.create(utilisateur_ptr=utilisateur)
# vendeur = Vendeur.objects.create(utilisateur_ptr=utilisateur)
# # agent = Agent.objects.create(utilisateur_ptr=utilisateur)

# # besoin de cette ligne pour créer une instance enfant à partir d'une instance parent
# # ca fait 17 ans que des gens demandent que ce soit fait automatiquement mdr
# # https://code.djangoproject.com/ticket/7623
# acheteur.__dict__.update(utilisateur.__dict__)
# vendeur.__dict__.update(utilisateur.__dict__)
# # agent.__dict__.update(utilisateur.__dict__)

# utilisateur, acheteur, vendeur

In [ ]:
x = Utilisateur(
    nom="Bambelle", prenom="Larry", telephone="+33158350027", email="bambelle.larry@youhou.fr"
)
# find if the user already exists
if Utilisateur.objects.

agence.models.Utilisateur.DoesNotExist()

In [4]:
Utilisateur.objects.all()

<QuerySet [<Utilisateur: Larry Bambelle  (bambelle.larry@youhou.fr, +33158350027)>]>

In [5]:
utilisateur

<Utilisateur: Larry Bambelle  (bambelle.larry@youhou.fr, +33158350027)>

In [6]:
acheteur = Acheteur.objects.create(utilisateur=utilisateur)

In [8]:
str(acheteur)

'Larry Bambelle  (bambelle.larry@youhou.fr, +33158350027)'

In [15]:
acheteur

<Acheteur: Larry Bambelle  (bambelle.larry@youhou.fr, +33158350027)>

In [11]:
# Utilisateur.objects.get(pk=1)
# set(Acheteur.objects.select_related("utilisateur_ptr"))
len(Acheteur.objects.filter(email="a@2.com"))

1

In [4]:
set(Vendeur.objects.select_related("utilisateur_ptr"))

{<Vendeur: m m  (a@a3.com, +33787312819)>}